In [5]:
# Define testA to accept a function and call it with x
def testA(fn):
    x = 0
    fn(x)

# Define a custom function to pass into testA
def myFunction(num):
    print("The number is:", num)

# Call testA and pass myFunction as argument
testA(myFunction)


The number is: 0


In [ ]:
a = testA
print(type(a))

<class 'function'>


: 